### Configure Ollama with structured outputs ###

In [1]:
import os
import pandas as pd
import numpy as np
import logging
import time
from pathlib import Path

logger = logging.getLogger(__name__)

import ollama

# Import this module with autoreload
%load_ext autoreload
%autoreload 2
import llmt
from llmt.llmtools import Prompt
from llmt.llmtools import process_prompt
from llmt.openai import OpenAIModel, OpenAI
from llmt.openai import MentalHealth, OutpatientServices, InpatientServices, create_messages
from llmt.performance import Performance

In [12]:
print(pd.__version__)

2.2.3


In [2]:
os.environ.get('OLLAMA_HOST')

'host.docker.internal'

In [4]:
from ollama import Client
client = Client(
  host='http://ollama:11434',
  headers={'x-some-header': 'some-value'}
)
response = client.chat(model='gemma3:4b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])

In [5]:
response

ChatResponse(model='gemma3:4b', created_at='2025-05-01T15:59:55.270374073Z', done=True, done_reason='stop', total_duration=16051026032, load_duration=1685567146, prompt_eval_count=15, prompt_eval_duration=110740112, eval_count=637, eval_duration=14251630904, message=Message(role='assistant', content="That's a fantastic question, and one that's fascinated people for centuries! The short answer is: **it's due to a phenomenon called Rayleigh scattering.** Here's a more detailed explanation:\n\n**1. Sunlight and Colors:**\n\n* Sunlight appears white, but it's actually made up of *all* the colors of the rainbow. Think of a prism splitting light – that's how sunlight is composed of red, orange, yellow, green, blue, and violet.\n\n**2. The Atmosphere and Tiny Particles:**\n\n* The Earth's atmosphere is filled with tiny particles – mostly nitrogen and oxygen molecules, but also dust, water droplets, and other small things.\n\n**3. Rayleigh Scattering:**\n\n* When sunlight enters the atmosphere

In [2]:
# Directories and files
data_dir = os.path.join(os.environ.get('HOME'), 'home_data', 'hcp')

test_file_name = 'hcp-alldata-250413.parquet'
test_file = os.path.join(data_dir, test_file_name)
df_all = pd.read_parquet(test_file)
# Filter the labeled data
df_train = df_all.loc[df_all['dset'] == 'train'].\
                reset_index(drop=True).\
                astype({'mental_health': int,
                        'inpatient': int,
                        'outpatient': int})
display(df_train.head())
print(df_train.shape)

,id,name,description,mental_health,inpatient,outpatient,dset
0,431643-07,Actriv,Provider of healthcare staffing services based...,2,0,0,train
1,310749-31,Alima,Operator of a non-governmental organization in...,0,0,0,train
2,162054-28,Apothecare,Provider of pharmacy services intended to prov...,0,0,0,train
3,597285-28,April Health (Clinics/Outpatient Services),Provider of mental health services intended to...,1,0,1,train
4,373978-90,Arise Child and Family Service,Operator of independent living centers caterin...,2,0,0,train


(187, 7)
